<a href="https://colab.research.google.com/github/artem-konevskikh/random-colab-notebooks/blob/main/cut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contrastive Unpaired Translation (CUT)


Made by [Artem Konevskikh](https://aiculedssul.net/)

[Original repository](https://github.com/taesungp/contrastive-unpaired-translation)

In [ ]:
#@title Mound Google Drive
#@markdown Connect Google Drive to save the converted point cloud
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title Install
import os
import shlex
!pip install dominate visdom GPUtil
!git clone https://github.com/taesungp/contrastive-unpaired-translation /content/drive/MyDrive/CUT
%cd /content/drive/MyDrive/CUT

In [ ]:
#@title Prepare Dataset
#@markdown First dataset folder
datasetA_dir = "/content/drive/MyDrive/uibk/mycelium-dataset" #@param {type: "string"}
#@markdown Second dataset folder
datasetB_dir = "/content/drive/MyDrive/Datasets/sat-dataset" #@param {type: "string"}
#@markdown CUT dataset folder
dataset_dir = "/content/drive/MyDrive/Datasets/cut-test1" #@param {type: "string"}

datasetA_dir = shlex.quote(datasetA_dir)
datasetB_dir = shlex.quote(datasetB_dir)
dataset_dir = shlex.quote(dataset_dir)

if not os.path.isdir(dataset_dir):
  !mkdir $dataset_dir 

!mkdir $dataset_dir/trainA $dataset_dir/trainB $dataset_dir/testA $dataset_dir/testB
!cp $datasetA_dir/* $dataset_dir/trainA/.
!cp $datasetB_dir/* $dataset_dir/trainB/.


In [ ]:
#@title Train
#@markdown Experiment name
name = "mycelium" #@param {type: "string"}
#@markdown Dataset folder
dataset_dir = "/content/drive/MyDrive/Datasets/cut-test1" #@param {type: "string"}
#@markdown Load size
load_size = 512 #@param {type: "integer"}
#@markdown Crop size
crop_size = 512 #@param {type: "integer"}
#@markdown Number of epochs
n_epochs = 200 #@param {type: "integer"}
#@markdown Frequency of saving checkpoints at the end of epochs
save_epoch_freq = 1  #@param {type: "integer"}
#@markdown Frequency of saving training results
update_html_freq = 100  #@param {type: "integer"}

name = shlex.quote(name)
dataset_dir = shlex.quote(dataset_dir)



!python train.py --dataroot $dataset_dir \
        --name $name \
        --CUT_mode CUT \
        --load_size $load_size \
        --crop_size $crop_size \
        --display_id 0 \
        --n_epochs $n_epochs \
        --save_epoch_freq $save_epoch_freq \
        --update_html_freq $update_html_freq

In [ ]:
#@title Generate images
#@markdown Experiment name
name = "mycelium" #@param {type: "string"}
#@markdown Dataset folder
dataset_dir = "/content/drive/MyDrive/Datasets/cut-test1" #@param {type: "string"}
#@markdown Direction
direction = 'AtoB' #@param ['AtoB', 'BtoA']
#@markdown Test images folder
test_imgs_dir = "/content/drive/MyDrive/Datasets/mycc" #@param {type: "string"}
#@markdown Results folder
results_dir = "/content/drive/MyDrive/Datasets/mycc_result" #@param {type: "string"}

test_imgs_dir = shlex.quote(test_imgs_dir)
!cp -f $test_imgs_dir/* $dataset_dir/testA/.
!cp -f $test_imgs_dir/* $dataset_dir/testB/.

results_dir = shlex.quote(results_dir)
if not os.path.isdir(results_dir):
  !mkdir $results_dir 

_, _, files = next(os.walk(test_imgs_dir))
num_test = len(files)

!python test.py --dataroot $dataset_dir --name $name --CUT_mode CUT --direction $direction --preprocess none --num_test $num_test
!cp /content/drive/MyDrive/CUT/results/$name/test_latest/images/fake_B/*.png $results_dir
!rm -rf /content/drive/MyDrive/CUT/results/$name/
!rm -rf $dataset_dir/testA/*
!rm -rf $dataset_dir/testB/*